In [ ]:
from gffutils.iterators import DataIterator
import os
import subprocess
import csv
import numpy as np
import re
import random
import glob
import pandas as pd
import re
import time
import h5py
import string
from random import choice
import sys
import time

In [ ]:
class Preprocess:

    ##############################################
    # Main Run function
    ##############################################
    def __init__(self, data_dir, HiChip_marks_list, num_seq_signals_excluding_histones,
                    HDF5_samples_num, using_fasta_seq, window_size):
        os.chdir(data_dir)
        start = time.process_time()
        
        # create class variable for window_size
        self.window_size = window_size
    
        # data size
        # Create gene regions file (includes indices of genes corresponding chromosome number)
        self.input_file_gff, self.output_file_gff = self.create_output_genes_file()
        print("Reference Genome: ", self.input_file_gff, "Output file: ", self.output_file_gff)
        self.genome_lenght, self.seq_regions_maize = self.genome_lenght(self.input_file_gff)

        # display genome length and Chromosome lengths
        print(self.genome_lenght, 'is the genome lenght')
        print(self.seq_regions_maize, 'are the lenghts of each chromosome by order')
        print(len(self.seq_regions_maize), 'are the total number of chromosomes')

        # get matrix for centers of anchors
        self.anchor_centers_matrix = self.get_centers_matrix(data_dir + "7205_SupTable_Loops.xlsx")
        print("Resulting centers matrix has shape: ", self.anchor_centers_matrix.shape)

        # process excel
        self.list_filename_data, self.list_filename_headers = self.process_excel_file(data_dir + "7205_SupTable_Loops.xlsx")

        print("filename_data: ")
        print(self.list_filename_data)
        print("filename_headers: ")
        print(self.list_filename_headers)

        # get positive samples HiChip_marks_data (creating the positive set with chr,s1,e1,s2,e2 window_size
        self.positive_set = self.generate_positive_set(self.anchor_centers_matrix)
        print("The specified window size is: ", self.window_size)
        print("Resulting window adjusted positive set matrix has shape: ", self.positive_set.shape)


        # get negative samples 
        self.negative_set = self.generate_negative_set(self.anchor_centers_matrix)

        # get combined dataset (combine negative samples with positives and shuffle)
        self.neg_and_pos_combined_set = self.combine_negative_positive_sets(self.positive_set, self.negative_set)
        
        # print generated matrix info
        print(self.positive_set.shape, 'is the positive set lenght')
        print(self.negative_set.shape, 'is the negative set lenght')
        print(self.neg_and_pos_combined_set.shape, 'is the combined set lenght')


        # create Histone samples of the data
        self.total_num_seq_signals = num_seq_signals_excluding_histones
        print(self.total_num_seq_signals, 'is total_num_seq')
        print(HDF5_samples_num, 'HDF5')
        
        # saving to hdf5 file
        self.h5_filename = self.data_to_h5(self.neg_and_pos_combined_set,
                                             self.total_num_seq_signals, HDF5_samples_num,
                                             using_fasta_seq )
        print('Data Preprocessing Finished.')
        print('Code execution time:', time.process_time() - start)
        
        
    ##############################################
    # Class Functions
    ##############################################
    def replace_chr_text_in_file(self, filename):
        # Read in the file
        with open(filename, 'r') as file:
            file_content = file.read()
        # Replace the target string
        pattern = re.compile("chr", re.IGNORECASE)
        file_content = pattern.sub("", file_content)

        # Write the file out again
        with open(filename, 'w') as file:
            file.write(file_content)

    # Calculate file length
    def reg_file_len(self, filename):
        with open(filename) as f:
            for i, l in enumerate(f):
                pass
        return i + 1
    
    def create_output_genes_file(self):
        if len(glob.glob("*.gff")) > 0:
            input_file_gff = glob.glob("*.gff")[0]
        elif len(glob.glob("*.gff3")) > 0:
            input_file_gff = glob.glob("*.gff3")[0]
        else:
            print("no GFF file found")

        if not os.path.exists("output_genes.txt"):
            with open(str("output_genes.txt"), 'w') as fout:
                for feature in DataIterator(str(input_file_gff)):
                    if feature.featuretype == "gene":
                        fout.write(str(feature[0]) + '\t' + str(feature[3]) + '\t' + str(feature[4]) + '\t' +
                                   (feature[8]["gene_id"])[0] + '\n')
            self.replace_chr_text_in_file(str("output_genes.txt"))

        output_file_gff = "output_genes.txt"
        return input_file_gff, output_file_gff
    
    ##############################################
    # Function: Calculate the lenght of the genome using the GFF file
    ##############################################
    #safe cast
    def safe_cast(self, val, to_type, default=None):
        try:
            return to_type(val)
        except (ValueError, TypeError):
            return default

    def genome_lenght(self, input_file_gff):
        seq_regions_maize_tmp = []
        list_order = []
        with open(input_file_gff) as f_original_gff:
            content_original_gff = f_original_gff.readlines()

        for hashtag_row in range(self.reg_file_len(input_file_gff)):
            if content_original_gff[hashtag_row].startswith('##'):
                if content_original_gff[hashtag_row].startswith('##sequence-region'):
                    temp_region = content_original_gff[hashtag_row].split()
                    if type(self.safe_cast(temp_region[1], int)) == int or str(temp_region[1]).startswith("chr"):
                        seq_regions_maize_tmp.append(self.safe_cast(temp_region[3], int))
                        list_order.append(self.safe_cast(temp_region[1], int) - 1)
            else:
                break

        seq_regions_maize = [None] * len(seq_regions_maize_tmp)
        for c, elem in enumerate(list_order):
            seq_regions_maize[elem] = seq_regions_maize_tmp[c]

        return sum(seq_regions_maize), seq_regions_maize
    
    
    ##############################################
    # Function: get centers matrix for anchors
    # @author : Defne
    ##############################################
    def get_centers_matrix(self, dataset_path):

        # Load spreadsheet: xlsx
        xlsx = pd.ExcelFile(dataset_path)
        all_sheet_dfs = []
        all_names = []
        #process the sheet names as necessary
        for name in xlsx.sheet_names[1:]:
            sheet = pd.read_excel(dataset_path, sheet_name = name)
            sheet_temp = sheet.drop(labels = [0,1,2,3,4], axis = 0)
            columns_to_keep = [x for x in range(sheet_temp.shape[1]) if x in [0,1,2,3,4]]
            sheet_temp = sheet_temp.iloc[:, columns_to_keep]
            all_sheet_dfs.append(sheet_temp)
            all_names.append(name)


        #get center matrices as dfs
        center_dfs = []
        for sheet_df in all_sheet_dfs:
            center_df = self.get_anchor_centers(sheet_df)
            center_dfs.append(center_df)

        #create a matrix from centered data and return that
        centers_matrix = pd.concat(center_dfs)
        centers_matrix_np = np.array(centers_matrix)    #return matrix as numpy

        return centers_matrix_np
    
    
    ##############################################
    # Helper function: get anchor centers
    # @author: Defne
    ##############################################
    def get_anchor_centers(self, dataset_df):

        center1_list = []
        center2_list = []
        chr_no_list = dataset_df.iloc[:,0].tolist()

        for index, row in dataset_df.iterrows():
            center1_list.append(int((row.iloc[1] + row.iloc[2]) / 2))
            center2_list.append(int((row.iloc[3] + row.iloc[4]) / 2))

        centers_matrix_column_list = list(zip(chr_no_list, center1_list, center2_list))

        anchor_centers_matrix = pd.DataFrame(centers_matrix_column_list)
        print(anchor_centers_matrix[1:5])
        return anchor_centers_matrix
    
    ##############################################
    # Function: process excel file
    # @author: Defne
    ##############################################
    def process_excel_file(self, dataset_path):
        #get assay data from excel according to leaf names
        #and create lists of names and headers
        #output: two list objects for headers and names
        #input: an xlsx file
        #output: list - filename_data, list - filename_header

        # Load spreadsheet: xls
        xlsx = pd.ExcelFile(dataset_path)

        #get sheet names and headers
        list_filename_data = []
        list_filename_headers = []

        #process the sheet names as necessary
        for name in xlsx.sheet_names[1:]:
            d_mark = name.split('_')[0]
            h_mark = d_mark + "_2500_headers.txt"
            d_mark = d_mark + "_2500_data.txt"

            if not os.path.isfile(d_mark) or not os.path.isfile(h_mark):
                d = pd.read_excel(dataset_path, sheet_name = name)
                h = d.iloc[4].tolist()
                h = h[0:5]
                d = d.drop(range(0,5))

                columns_to_keep = [x for x in range(d.shape[1]) if x in [0,1,2,3,4]]
                d = d.iloc[:, columns_to_keep]

                #get header names and create the headers.txt file
                d.columns = h

                #save the data and headers in .txt files
                np.savetxt(d_mark, d.values, delimiter = "\t ", fmt = '%s')

            list_filename_data.append(d_mark)
            list_filename_headers.append(h_mark)

        return list_filename_data, list_filename_headers
    
    ##############################################
    # Function: Generate positive labled set using
    # HiChip marks data (also does the window_size 
    # adjustment)
    # @author: Defne
    ##############################################
    def generate_positive_set(self, anchor_centers_matrix):
    
        #for convinience - change data to dataframe
        centers_df = pd.DataFrame(anchor_centers_matrix)

        s1_list = []
        e1_list = []
        s2_list = []
        e2_list = []
        chr_no_list = centers_df.iloc[:,0].tolist()
        #window_size = 2500

        for index, row in centers_df.iterrows():
            s1_list.append(int(row.iloc[1] - int(self.window_size/2)))
            e1_list.append(int(row.iloc[1] + int(self.window_size/2)))

            s2_list.append(int(row.iloc[2] - int(self.window_size/2)))
            e2_list.append(int(row.iloc[2] + int(self.window_size/2)))

        matrix_column_list = list(zip(chr_no_list, s1_list, e1_list, s2_list, e2_list))
        
        window_size_adjusted_positive_matrix = pd.DataFrame(matrix_column_list).to_numpy()

        # create labels 
        positive_labels = np.full(len(adjusted_to_original_matrix), 1)
        positive_labels = positive_labels.reshape(len(positive_labels), -1)
        window_size_adjusted_positive_matrix = np.append(window_size_adjusted_positive_matrix, positive_labels, axis=1)
        
        print('positive set first rows', window_size_adjusted_positive_matrix[0:5])
        
        return window_adjusted_pos_matrix
    
    ##############################################
    # Function: Adjusting dataset from centers 
    # data to s1,e1,s2,e2 window
    # @author: Defne
    ##############################################
    def adjust_negative_set_to_original_window(self, centers_matrix):

        #for convinience - change data to dataframe
        centers_df = pd.DataFrame(centers_matrix)

        s1_list = []
        e1_list = []
        s2_list = []
        e2_list = []
        chr_no_list = centers_df.iloc[:,0].tolist()
        #window_size = 2500

        #loop through the center coordinates and adjust s1,e1,s2,e2 coordinates
        for index, row in centers_df.iterrows():
            s1_list.append(int(row.iloc[1] - int(self.window_size/2)))
            e1_list.append(int(row.iloc[1] + int(self.window_size/2)))

            s2_list.append(int(row.iloc[2] - int(self.window_size/2)))
            e2_list.append(int(row.iloc[2] + int(self.window_size/2)))

        matrix_column_list = list(zip(chr_no_list, s1_list, e1_list, s2_list, e2_list))

        window_size_adjusted_negative_matrix = pd.DataFrame(matrix_column_list).to_numpy()
        #change matrix to numpy
        #adjusted_to_original_matrix = adjusted_to_original_matrix.to_numpy()

        #label with all negatives (zeros)
        negative_labels = np.full(len(adjusted_to_original_matrix), 0)
        negative_labels = negative_labels.reshape(len(negative_labels), -1)
        window_size_adjusted_negative_matrix = np.append(window_size_adjusted_negative_matrix, negative_labels, axis=1)
        print('negative set first rows', window_size_adjusted_negative_matrix[0:5])
        
        #print(adjusted_to_original_matrix)

        return window_size_adjusted_negative_matrix
    
    ##############################################
    # Function: generate negative set with window 
    # adjustment
    # @author: Luca
    ##############################################

    def generate_negative_set(self,anchor_centers_matrix):
        anchor_centers_matrix = anchor_centers_matrix[np.unique(anchor_centers_matrix[:, [0, 1, 2]], return_index=True,axis=0)[1]]
        chr_column = anchor_centers_matrix[:,0]
        max_chr = max(chr_column)
        negative_set = np.zeros([0,3], dtype = int)
        min_dist = 0
        max_dist = 300000000#200000
        stop = 100000#000
        # go on here 
        #anchor_list = np.append(anchor_centers_matrix[:, [0, 1]], anchor_centers_matrix[:, [0, 2]])
        print('ANCHOR CENTER MATRIX:', anchor_centers_matrix)
        
        #if not os.path.exists("negative_training_set.txt"):
        for i in range(1,max_chr+1):
            print('Handling chr ', i)
            submatrix = anchor_centers_matrix[anchor_centers_matrix[:,0] == i]
            max_chr_idx = len(submatrix[:,0])
            print('-------')
            submatrix_permutated = submatrix.copy()
            submatrix_permutated[:,2] = 0
            for j in range(0,max_chr_idx):        
                check_distance = False
                es = 0
                while check_distance == False:
                    if es > stop:
                        sys.exit("Error: Max distance is too small.")
                    # choose a random integer excluding j - no overlap with positive set.
                    #random_index2 = choice([x for x in range(0,max_chr_idx) if x not in [j]])
                    lower_index = random.randint(2,max_chr_idx)#(2,50)
                    upper_index = random.randint(2,max_chr_idx)#(2,50)
                    random_index3 = choice([x for x in range(max(0,j-lower_index),min(j+upper_index,max_chr_idx)) if x not in [j]])
                    anchor1 = submatrix[j,1]
                    anchor2 = submatrix[random_index3,2]
                    random_shift = choice([x for x in range(-25,25) if x not in range(-5,5)])
                    random_shift2 = choice([x for x in range(-25,25) if x not in range(-5,5)])
                    distance = abs(anchor2 - anchor1)
                    if distance > min_dist and distance < max_dist:                        
                        submatrix_permutated[j,2]= anchor2
                        submatrix_permutated[j,2] += random_shift
                        submatrix_permutated[j,1] += random_shift2
                        check_distance = True   
                    else:
                        es = es + 1            
            negative_set = np.concatenate([negative_set,submatrix_permutated])        
        print('negative_set:\n', negative_set[0:10])
        np.savetxt('negative_training_set.txt',negative_set, fmt = '%i', delimiter = '\t')        
        #negative_samples = negative_set
        #size = negative_set.shape
        print('dimension of the created negative set:', negative_set.shape)
        window_adjusted_negative_set = self.adjust_negative_set_to_original_window(negative_set)
        np.savetxt('negative_training_set.txt',window_adjusted_negative_set, fmt = '%i', delimiter = '\t')

        return window_adjusted_negative_set
    
    ##############################################
    # Function: Combine negative samples with positives 
    # and shuffle
    # Train & Test Samples
    ##############################################
    def combine_negative_positive_sets(self, positive_set, negative_set):
        print("Combining positive and negative set.")
        print('received positive set:', positive_set[0:5])
        print('received negative :', negative_set[0:5])
        combined_set = np.concatenate((positive_set, negative_set))
        # shuffle the negatives and positives
        np.random.shuffle(combined_set)
        print("len Shuffle:", len(combined_set))
        np.savetxt('training_set_shuffled.txt',combined_set, fmt = '%i', delimiter = '\t')
        return combined_set
    
    ##############################################
    # process_FASTA_vector
    ##############################################
    def process_FASTA_vector(self, region_s, region_e, chr_num_in_line, retrieve_FASTA_matrix):

        FASTA_sequences_filename = str(region_s) + "_" + str(region_e) + ".txt"

        if not os.path.exists("FASTA_sequences/" + chr_num_in_line + "/" + FASTA_sequences_filename):
            # create FASTA per_base_cov directory if not exists
            if not os.path.exists("FASTA_sequences"):
                try:
                    os.mkdir("FASTA_sequences")
                except OSError:
                    print("Creation of the directory failed -- FASTA_sequences")

            if not os.path.exists("FASTA_sequences/" + chr_num_in_line):
                try:
                    os.mkdir("FASTA_sequences/" + chr_num_in_line)
                except OSError:
                    print("Creation of the directory failed -- " + "FASTA_sequences/" + chr_num_in_line)

            FASTA_sequences_filename = str(region_s) + "_" + str(region_e) + ".txt"

            if not os.path.exists("FASTA_sequences/" + chr_num_in_line + "/" + FASTA_sequences_filename):
                with open('temp_bash.sh', 'w') as the_file:
                    the_file.write(
                        "bedtools getfasta -fi Zea_mays_B73_v4.fasta -bed temp_region.txt > FASTA_sequences/" + chr_num_in_line + "/" + FASTA_sequences_filename)
                subprocess.call(["sh", "./temp_bash.sh"])
            
        if retrieve_FASTA_matrix and os.path.exists(
                "FASTA_sequences/" + chr_num_in_line + "/" + FASTA_sequences_filename):
            FASTA_matrix = np.zeros((4, 2500))  # ACGT
            print(chr_num_in_line)
            print(FASTA_sequences_filename)
            fp = open("FASTA_sequences/" + chr_num_in_line + "/" + FASTA_sequences_filename)
            for i, line in enumerate(fp):
                if i == 1:
                    line_character_count = 0
                    for c in line:
                        # print c
                        if c == "A":
                            FASTA_matrix[0][line_character_count] = 1
                        if c == "C":
                            FASTA_matrix[1][line_character_count] = 1
                        if c == "G":
                            FASTA_matrix[2][line_character_count] = 1
                        if c == "T":
                            FASTA_matrix[3][line_character_count] = 1
                        if c == "N":
                            FASTA_matrix[0][line_character_count] = 0.25
                            FASTA_matrix[1][line_character_count] = 0.25
                            FASTA_matrix[2][line_character_count] = 0.25
                            FASTA_matrix[3][line_character_count] = 0.25
                        line_character_count += 1
            fp.close()

            return FASTA_matrix


    ##############################################
    # create Matrix
    ##############################################
    def create_matrix(self, chr_num, region_s, region_e, using_fasta_seq):

        with open('temp_region.txt', 'w') as the_file:
            the_file.write(str(chr_num) + "\t" + str(region_s) + "\t" + str(region_e))

        #print("subprocess call will start")

        # empty the temp_bash.sh file
        open('temp_bash.sh', 'w').close()

        subprocess_bedtools_job_tracker_counter = 0

        if not os.stat("temp_bash.sh").st_size == 0:
            for subprocess_count in range(subprocess_bedtools_job_tracker_counter):
                with open('temp_bash.sh', 'a') as the_file:
                    letter = string.ascii_lowercase[subprocess_count]
                    the_file.write("wait $" + letter + "\n" "echo \"job " + letter + " returned $?\" \n")
                    the_file.close()

        # subprocess call
        subprocess.call(["sh", "./temp_bash.sh"])
        #print("subprocess call ended")

        # generate sequence
        if using_fasta_seq:
            self.process_FASTA_vector(region_s, region_e, chr_num, retrieve_FASTA_matrix=False)

        final_matrix = []

        final_matrix = np.array(final_matrix, dtype=int)

        # get ACGT sequence
        if using_fasta_seq:
            FASTA_matrix = self.process_FASTA_vector(region_s, region_e, chr_num, retrieve_FASTA_matrix=True)
            if len(final_matrix) == 0:
                final_matrix = FASTA_matrix
            else:
                final_matrix = np.array(final_matrix, dtype=int)
                final_matrix = np.vstack((final_matrix, FASTA_matrix))

        final_matrix = np.array(final_matrix, dtype=int)
        #print('FINAL_CREATE_MATRIX MATRIX:', final_matrix)
        #print(final_matrix.shape, "final_matrix shape")
        return final_matrix


    ##############################################
    # Create hdf5 file of the data (sequence information and labels)
    ##############################################
    def data_to_h5(self, data_matrix_with_neg_and_pos,
                                           total_num_seq_signals, HDF5_samples_num,
                                           using_fasta_seq):

        h5_filename = "regions_labels_rand_neg_" + str(HDF5_samples_num) + "2500" +".h5"
        print('total: ', total_num_seq_signals)
        #print('HDF5: ', HDF5_samples_num ) # this is 202344
        print('using_fasta_seq: ', using_fasta_seq)
        if not 1>2:#os.path.isfile(h5_filename):
            hf = h5py.File(h5_filename, 'w')
            hf.create_dataset('re_region_matrices', shape=(HDF5_samples_num, total_num_seq_signals, 2500), dtype="int",
                              data=None)
            hf.create_dataset('interactions_region_matrices', shape=(HDF5_samples_num, total_num_seq_signals, 2500),
                              dtype="int", data=None)

            # get labels
            print("retrieving labels")
            labels = data_matrix_with_neg_and_pos[:, -1]
            labels = labels.astype(int)
            hf.create_dataset('labels', data=labels[:HDF5_samples_num])

            # re_region_matrices = []
            for data_count, data_sample in enumerate(data_matrix_with_neg_and_pos[:HDF5_samples_num]):
                #print(data_count, " retrieving regulatory element region matrices")
                chr_num = data_sample[0]
                #print('chr_num:',chr_num)
                re_region_s = data_sample[1]
                #print('re_region_s:',re_region_s)
                re_region_e = data_sample[2]
                #print('re_region_e:',re_region_e)

                re_region_matrix = self.create_matrix(str(chr_num), re_region_s, re_region_e,using_fasta_seq)
                #print('re_region_matrix: ', re_region_matrix)
                with h5py.File(h5_filename, 'a') as hf:
                    #print('hf:', hf)
                    hf["re_region_matrices"][data_count] = re_region_matrix

            for data_count, data_sample in enumerate(data_matrix_with_neg_and_pos[:HDF5_samples_num]):
                #print(data_count, " retrieving interactions region matrices")
                chr_num = data_sample[0]
                in_region_s = data_sample[3]
                in_region_e = data_sample[4]

                interactions_region_matrix = self.create_matrix(str(chr_num), in_region_s, in_region_e,using_fasta_seq)
                with h5py.File(h5_filename, 'a') as hf:
                    hf["interactions_region_matrices"][data_count] = interactions_region_matrix

            hf.close()

            print("done creating HDF5 file")

        return h5_filename

In [ ]:
## Call init of the class
Preprocess("../data/", "k4me_2500,k27me_2500", 4, 202344, True, window_size=2500)